# Collecting the data
First we have to get all the needed data. We will use web3py with Pancake bet contract (0x0E3A8078EDD2021dadcdE733C6b4a86E51EE8f07) and requests with bscscan API.

Add needed modules folders:

In [ ]:
import sys
import os

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

if module_path + '/src' not in sys.path:
    sys.path.append(module_path + '/src')

# os.chdir("..")

print(os.getcwd())

## Getting active players
Use get_active_players.py main() with start and end time. It may be required to change the main function loop to iterate through more past transactions.
Also change CURRENT_BLOCK_NUMBER to the current one - to start checking transactions before this block.

In [ ]:
import datetime

import get_active_players

# The same timeframe as for the training set
time_from_active_players = int(datetime.datetime(2022, 10, 28, 0, 0).timestamp())
time_to_active_players = int(datetime.datetime(2022, 12, 28, 0, 0).timestamp())
active_players_file = f'../data/active_players_2months.csv'

get_active_players.make_active_players_file(time_from_active_players, time_to_active_players, 70, active_players_file)